## PPI Graph Features

#### Libraries

In [ ]:
import pickle
import networkx as nx
import os
# import pandas as pd
import matplotlib.pyplot as plt
import statistics

#### Variables

In [ ]:
# Name of the files to load
PPI_list_variable_corrected_updated = "data/PPI_of_interest_corrected updated"

# Name of the files to save/load
Graph_of_PPI_file_name = "results/PPIGraph/Graph_of_PPI"
Graph_of_PPI_LCC_file_name = "results/PPIGraph/Graph_of_PPI_LCC"
Sorted_Degrees_figure_name = "results/PPIGraph/Sorted Degrees of the PPI Network"
Degrees_Histogram_Visualization = "results/PPIGraph/Degrees Histogram Visualization"
LCC_average_shortest_path_file_name = "results/PPIGraph/LCC_average_shortest_path"
LCC_clustering_coefficient_file_name = "results/PPIGraph/LCC_clustering_coefficient"
LCC_Density_file_name = "results/PPIGraph/LCC_Density"
LLC_Global_Efficiency_file_name = "results/PPIGraph/LLC_Global_Efficiency"
LLC_Local_Efficiency_file_name = "results/PPIGraph/LLC_Local_Efficiency"

# other strings we'll use
First_gene_symbol_indicator = "Official Symbol Interactor A"
Second_gene_symbol_indicator = "Official Symbol Interactor B"

#### Graph Features

In [ ]:
with open(PPI_list_variable_corrected_updated, "rb") as file:
    PPI = pickle.load(file)


def Load_or_Save_Variable_from_nx_method(file_name, Graph, nx_method):
    if not os.path.exists(file_name):
        Variable = nx_method(Graph)
        with open(file_name, "wb") as file:
            pickle.dump(Variable, file)
    else:
        with open(file_name, "rb") as file:
            Variable = pickle.load(file)
    return Variable


if not (os.path.exists(Graph_of_PPI_file_name)
        and os.path.exists(Graph_of_PPI_LCC_file_name)):
    Graph_of_PPI = nx.from_pandas_edgelist(
        PPI, First_gene_symbol_indicator, Second_gene_symbol_indicator)
    with open(Graph_of_PPI_file_name, "wb") as file:
        pickle.dump(Graph_of_PPI, file)

    Connected_Components = nx.connected_components(Graph_of_PPI)
    largest_cc = max(Connected_Components, key=len)
    Graph_of_PPI_LCC = nx.subgraph(Graph_of_PPI, largest_cc).copy()
    with open(Graph_of_PPI_LCC_file_name, "wb") as file:
        pickle.dump(Graph_of_PPI, file)

else:
    with open(Graph_of_PPI_file_name, "rb") as file:
        Graph_of_PPI = pickle.load(file)
    with open(Graph_of_PPI_LCC_file_name, "rb") as file:
        Graph_of_PPI_LCC = pickle.load(file)

print("Features of the initial PPI:\n" + str(Graph_of_PPI) + "\n")
print("Features of the LCC from the initial PPI:\n" + str(Graph_of_PPI_LCC) + "\n")

degress_of_LCC_dictionary = dict(Graph_of_PPI.degree())
degrees_of_LCC_dictionary_sorted = \
    dict(sorted(degress_of_LCC_dictionary.items(), key=lambda item: item[1],
                reverse=True))
degrees_of_LCC_values = list(degress_of_LCC_dictionary.values())
degrees_of_LCC_keys = list(degress_of_LCC_dictionary.keys())
mean_degree_of_LCC = statistics.mean(degrees_of_LCC_values)
median_degree_of_LCC = statistics.median(degrees_of_LCC_values)
sorted_degrees_of_LCC_values = list(degrees_of_LCC_dictionary_sorted.values())
sorted_degrees_of_LCC_keys = list(degrees_of_LCC_dictionary_sorted.keys())

# # Figures Section
# plt.figure(figsize=(12, 9))
# plt.subplot(1, 2, 1)
# plt.bar(range(len(sorted_degrees_of_LCC_values)), sorted_degrees_of_LCC_values)
# plt.xlabel("Nodes")
# plt.ylabel("Nodes Degrees")
# plt.subplot(1, 2, 2)
# plt.bar(sorted_degrees_of_LCC_keys, sorted_degrees_of_LCC_values)
# plt.xticks(rotation = 45, ha ='right')
# plt.ylabel("Nodes Deegres")
# plt.xlim(-0.6, 15.6)
# plt.suptitle("Sorted Degrees of the PPI network", fontsize = 20)
# plt.savefig(Sorted_Degrees_figure_name, dpi = 500)

plt.figure(figsize=(12, 9))
plt.subplot(1, 2, 1)
plt.hist(degrees_of_LCC_values,
         bins=range(min(degrees_of_LCC_values), max(degrees_of_LCC_values) + 1))
plt.yscale('log')
plt.xlabel("Degrees Values")
plt.ylabel("Degrees Frequency (log scale)")
plt.subplot(1, 2, 2)
plt.hist(degrees_of_LCC_values,
         bins=range(min(degrees_of_LCC_values), max(degrees_of_LCC_values) + 1))
plt.xscale('log')
plt.yscale('log')
plt.xlabel("Degrees Values (log scale)")
plt.ylabel("Degrees Frequency (log scale)")
plt.suptitle("Degrees Histogram of the PPI Network", fontsize=20)
plt.savefig(Degrees_Histogram_Visualization, dpi=500, bbox_inches='tight')

plt.figure(figsize=(12, 9))
plt.subplot(1, 2, 1)
plt.hist(degrees_of_LCC_values,
         bins=range(min(degrees_of_LCC_values), max(degrees_of_LCC_values) + 1))
plt.xlabel("Degrees Values")
plt.ylabel("Degrees Frequency (log scale)")
plt.ylim(0, 1700)
plt.subplot(1, 2, 2)
plt.hist(degrees_of_LCC_values,
         bins=range(min(degrees_of_LCC_values), max(degrees_of_LCC_values) + 1))
plt.xlim(0.3, 15.3)
plt.xlabel("Degrees Values (log scale)")
plt.ylabel("Degrees Frequency (log scale)")
plt.suptitle("Degrees Histogram of the PPI Network", fontsize=20)

LCC_average_shortest_path = \
    Load_or_Save_Variable_from_nx_method(LCC_average_shortest_path_file_name,
                                         Graph_of_PPI_LCC,
                                         nx.average_shortest_path_length)
LCC_clustering_coefficient = \
    Load_or_Save_Variable_from_nx_method(LCC_clustering_coefficient_file_name,
                                         Graph_of_PPI_LCC,
                                         nx.clustering)
LCC_clustering_coefficient_Values = list(LCC_clustering_coefficient.values())
plt.plot(sorted(LCC_clustering_coefficient_Values, reverse=True))
LCC_mean_clustering_coefficient = statistics.mean(LCC_clustering_coefficient_Values)
LCC_median_clustering_coefficient = statistics.median(LCC_clustering_coefficient_Values)

LCC_Density = \
    Load_or_Save_Variable_from_nx_method(LCC_Density_file_name,
                                         Graph_of_PPI_LCC,
                                         nx.density)
LLC_Global_Efficiency = \
    Load_or_Save_Variable_from_nx_method(LLC_Global_Efficiency_file_name,
                                         Graph_of_PPI_LCC,
                                         nx.global_efficiency)
